In [1]:
library(RPostgreSQL)
library(memoise)
library(xgboost)
library(caret)
library(GA)
library(tidyverse)

Loading required package: DBI
Loading required package: lattice
Loading required package: ggplot2
Loading required package: foreach
Loading required package: iterators
Package 'GA' version 3.0.2
Type 'citation("GA")' for citing this R package in publications.
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ tibble  1.3.4     ✔ purrr   0.2.4
✔ tidyr   0.7.2     ✔ dplyr   0.7.4
✔ readr   1.1.1     ✔ stringr 1.2.0
✔ tibble  1.3.4     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::accumulate() masks foreach::accumulate()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()
✖ purrr::lift()       masks caret::lift()
✖ dplyr::slice()      masks xgboost::slice()
✖ purrr::when()       masks foreach::when()


In [2]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [3]:
full_data <- dbGetQuery(con, "select * from merged_data")

In [4]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE

In [5]:
head(full_data)
names(full_data)
dim(full_data)

hadm_id,icustay_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_chloride_flag,lab_chloride_first,lab_chloride_min,lab_chloride_max,lab_chloride_abnormal,lab_ph_flag,lab_ph_first,lab_ph_min,lab_ph_max,lab_ph_abnormal
125078,201220,66690,MICU,2106-04-27 01:47:50,2106-05-01 11:25:46,1,62.67646,1,NA,⋯,1,123,123,128,1,1,7.45,7.42,7.45,1
151232,215842,11663,MICU,2188-02-14 01:48:15,2188-02-15 19:02:48,1,86.76186,1,NA,⋯,1,105,105,105,0,0,NA,NA,NA,NA
164444,234312,86645,SICU,2165-06-22 01:47:16,2165-07-07 13:55:20,1,56.08904,1,NA,⋯,1,108,107,112,1,1,7.49,7.49,7.49,1
146726,289157,10304,MICU,2156-06-23 14:26:00,2156-06-30 09:26:00,1,45.91093,1,2156-06-25,⋯,1,100,100,105,0,1,7.44,7.44,7.44,1
160170,211964,94534,MICU,2160-03-05 14:23:19,2160-03-06 22:48:41,1,59.38693,1,NA,⋯,1,103,103,103,0,0,NA,NA,NA,NA
112553,230173,31544,MICU,2140-01-30 20:39:25,2140-02-02 18:41:39,1,300.00345,1,NA,⋯,1,118,113,118,1,0,NA,NA,NA,NA


[1] "hadm_id"                        "icustay_id"                    
  [3] "subject_id"                     "first_careunit"                
  [5] "intime"                         "outtime"                       
  [7] "angus"                          "age"                           
  [9] "icu_order"                      "echo_time"                     
 [11] "echo"                           "gender"                        
 [13] "weight"                         "saps"                          
 [15] "sofa"                           "elix_score"                    
 [17] "vent"                           "vaso"                          
 [19] "weekday"                        "mort_28_day"                   
 [21] "icd_chf"                        "icd_afib"                      
 [23] "icd_renal"                      "icd_liver"                     
 [25] "icd_copd"                       "icd_cad"                       
 [27] "icd_stroke"                     "icd_malignancy"                
 [29] "vs_temp_flag"                   "vs_temp_first"                 
 [31] "vs_temp_min"                    "vs_temp_max"                   
 [33] "vs_heart_rate_flag"             "vs_heart_rate_first"           
 [35] "vs_heart_rate_min"              "vs_heart_rate_max"             
 [37] "vs_map_flag"                    "vs_map_first"                  
 [39] "vs_map_min"                     "vs_map_max"                    
 [41] "vs_cvp_flag"                    "vs_cvp_first"                  
 [43] "vs_cvp_min"                     "vs_cvp_max"                    
 [45] "lab_hemoglobin_flag"            "lab_hemoglobin_first"          
 [47] "lab_hemoglobin_min"             "lab_hemoglobin_max"            
 [49] "lab_hemoglobin_abnormal"        "lab_wbc_flag"                  
 [51] "lab_wbc_first"                  "lab_wbc_min"                   
 [53] "lab_wbc_max"                    "lab_wbc_abnormal"              
 [55] "lab_lactate_flag"               "lab_lactate_first"             
 [57] "lab_lactate_min"                "lab_lactate_max"               
 [59] "lab_lactate_abnormal"           "lab_po2_flag"                  
 [61] "lab_po2_first"                  "lab_po2_min"                   
 [63] "lab_po2_max"                    "lab_po2_abnormal"              
 [65] "lab_troponin_flag"              "lab_troponin_first"            
 [67] "lab_troponin_min"               "lab_troponin_max"              
 [69] "lab_troponin_abnormal"          "lab_potassium_flag"            
 [71] "lab_potassium_first"            "lab_potassium_min"             
 [73] "lab_potassium_max"              "lab_potassium_abnormal"        
 [75] "lab_creatinine_kinase_flag"     "lab_creatinine_kinase_first"   
 [77] "lab_creatinine_kinase_min"      "lab_creatinine_kinase_max"     
 [79] "lab_creatinine_kinase_abnormal" "lab_creatinine_flag"           
 [81] "lab_creatinine_first"           "lab_creatinine_min"            
 [83] "lab_creatinine_max"             "lab_creatinine_abnormal"       
 [85] "lab_pco2_flag"                  "lab_pco2_first"                
 [87] "lab_pco2_min"                   "lab_pco2_max"                  
 [89] "lab_pco2_abnormal"              "lab_bnp_flag"                  
 [91] "lab_bnp_first"                  "lab_bnp_min"                   
 [93] "lab_bnp_max"                    "lab_bnp_abnormal"              
 [95] "lab_bun_flag"                   "lab_bun_first"                 
 [97] "lab_bun_min"                    "lab_bun_max"                   
 [99] "lab_bun_abnormal"               "lab_bicarbonate_flag"          
[101] "lab_bicarbonate_first"          "lab_bicarbonate_min"           
[103] "lab_bicarbonate_max"            "lab_bicarbonate_abnormal"      
[105] "lab_platelet_flag"              "lab_platelet_first"            
[107] "lab_platelet_min"               "lab_platelet_max"              
[109] "lab_platelet_abnormal"          "lab_sodium_flag"               
[111] "lab_sodium_first"               "la

[1] 6162  124

In [6]:
feature_names <- full_data %>%
    names %>%
    keep(grepl("vs|lab|icd|age|gender|weight|saps|sofa|elix_score|vent|vaso", .)) %>%
    discard(grepl("vs", .) & grepl("flag", .)) %>%
    discard(grepl("min|max", .))
feature_names

[1] "age"                            "gender"                        
 [3] "weight"                         "saps"                          
 [5] "sofa"                           "elix_score"                    
 [7] "vent"                           "vaso"                          
 [9] "icd_chf"                        "icd_afib"                      
[11] "icd_renal"                      "icd_liver"                     
[13] "icd_copd"                       "icd_cad"                       
[15] "icd_stroke"                     "icd_malignancy"                
[17] "vs_temp_first"                  "vs_heart_rate_first"           
[19] "vs_map_first"                   "vs_cvp_first"                  
[21] "lab_hemoglobin_flag"            "lab_hemoglobin_first"          
[23] "lab_hemoglobin_abnormal"        "lab_wbc_flag"                  
[25] "lab_wbc_first"                  "lab_wbc_abnormal"              
[27] "lab_lactate_flag"               "lab_lactate_first"             
[29] "lab_lactate_abnormal"           "lab_po2_flag"                  
[31] "lab_po2_first"                  "lab_po2_abnormal"              
[33] "lab_troponin_flag"              "lab_troponin_first"            
[35] "lab_troponin_abnormal"          "lab_potassium_flag"            
[37] "lab_potassium_first"            "lab_potassium_abnormal"        
[39] "lab_creatinine_kinase_flag"     "lab_creatinine_kinase_first"   
[41] "lab_creatinine_kinase_abnormal" "lab_creatinine_flag"           
[43] "lab_creatinine_first"           "lab_creatinine_abnormal"       
[45] "lab_pco2_flag"                  "lab_pco2_first"                
[47] "lab_pco2_abnormal"              "lab_bnp_flag"                  
[49] "lab_bnp_first"                  "lab_bnp_abnormal"              
[51] "lab_bun_flag"                   "lab_bun_first"                 
[53] "lab_bun_abnormal"               "lab_bicarbonate_flag"          
[55] "lab_bicarbonate_first"          "lab_bicarbonate_abnormal"      
[57] "lab_platelet_flag"              "lab_platelet_first"            
[59] "lab_platelet_abnormal"          "lab_sodium_flag"               
[61] "lab_sodium_first"               "lab_sodium_abnormal"           
[63] "lab_chloride_flag"              "lab_chloride_first"            
[65] "lab_chloride_abnormal"          "lab_ph_flag"                   
[67] "lab_ph_first"                   "lab_ph_abnormal"

In [7]:
features <- full_data %>%
    select(!!!rlang::syms(feature_names)) %>%
    mutate(gender = as.integer(as.factor(gender)))
head(features)

age,gender,weight,saps,sofa,elix_score,vent,vaso,icd_chf,icd_afib,⋯,lab_platelet_abnormal,lab_sodium_flag,lab_sodium_first,lab_sodium_abnormal,lab_chloride_flag,lab_chloride_first,lab_chloride_abnormal,lab_ph_flag,lab_ph_first,lab_ph_abnormal
62.67646,2,74.3,25,5,5,1,0,0,0,⋯,0,1,160,1,1,123,1,1,7.45,1
86.76186,1,NA,13,1,10,0,0,0,0,⋯,0,1,139,0,1,105,0,0,NA,NA
56.08904,2,65.0,18,5,14,1,0,0,0,⋯,0,1,144,1,1,108,1,1,7.49,1
45.91093,2,NA,16,9,13,0,0,0,0,⋯,1,1,134,1,1,100,0,1,7.44,1
59.38693,2,91.4,13,3,22,0,0,0,0,⋯,1,1,138,0,1,103,0,0,NA,NA
300.00345,1,55.0,25,5,0,0,0,0,0,⋯,0,1,147,1,1,118,1,0,NA,NA


In [8]:
label <- full_data %>% pull(echo)
head(label)

[1] 0 0 0 1 0 0

In [9]:
cost <- function(string, x, y) {
    features_mtx <- x[, which(string == 1)] %>% data.matrix
    label <- y
    1:nrow(features_mtx) %>%
    createFolds(10) %>%
    map_dbl(function(index) {
        model <- xgboost(features_mtx[index, ], label[index],
                         params = list(objective = "binary:logistic"),
                         nrounds = 100, verbose = 0)
        pred <- predict(model, features_mtx[-index, ])
        ROCR::performance(ROCR::prediction(pred, label[-index]), "auc")@y.values %>% first
    }) %>% mean
}

In [13]:
cost(base::sample(0:1, nrow(features), replace = TRUE), features, label)

ERROR: Error in `[.data.frame`(x, , which(string == 1)): undefined columns selected


In [10]:
mcost <- memoise(cost)
is.memoised(mcost)

[1] TRUE

In [11]:
initialPop <- function(object, ...) {
    population <- sample(0:1, 
                         replace = TRUE, 
                         size = object@nBits * object@popSize, 
                         prob = c(0.9, 0.1))
    population <- matrix(population, 
                         nrow = object@popSize, 
                         ncol = object@nBits)
    return(population)
}

In [12]:
ga_results <- ga(type = "binary",
                 fitness = mcost,
                 x = features,
                 y = label,
                 min = 0, max = 1,
                 maxiter = 1,
                 popSize = 10,
                 population = initialPop,
                 nBits = ncol(features),
                 names = feature_names,
                 keepBest = TRUE,
                 parallel = 4)

In [ ]:
ga_results